# Librerías

In [67]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Directorio donde se guarda el modelo fine-tuned

In [68]:
INFER_DIR = "./outputs/m2m100_finetuned_lora_infer"
assert os.path.isdir(INFER_DIR), f"No existe el directorio de inferencia: {INFER_DIR}"

# Selección del dispositivo

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Recargar tokenizer y modelo desde disco

In [70]:
tokenizer_inf = AutoTokenizer.from_pretrained(
    INFER_DIR,
    local_files_only=True
)
model_inf = AutoModelForSeq2SeqLM.from_pretrained(
    INFER_DIR,
    local_files_only=True
).to(device)

# Configurar idiomas (requerido por M2M100)

In [71]:
# Mostrar todos los códigos de idioma disponibles
print(tokenizer_inf.lang_code_to_id.keys())

dict_keys(['af', 'am', 'ar', 'ast', 'az', 'ba', 'be', 'bg', 'bn', 'br', 'bs', 'ca', 'ceb', 'cs', 'cy', 'da', 'de', 'el', 'en', 'es', 'et', 'fa', 'ff', 'fi', 'fr', 'fy', 'ga', 'gd', 'gl', 'gu', 'ha', 'he', 'hi', 'hr', 'ht', 'hu', 'hy', 'id', 'ig', 'ilo', 'is', 'it', 'ja', 'jv', 'ka', 'kk', 'km', 'kn', 'ko', 'lb', 'lg', 'ln', 'lo', 'lt', 'lv', 'mg', 'mk', 'ml', 'mn', 'mr', 'ms', 'my', 'ne', 'nl', 'no', 'ns', 'oc', 'or', 'pa', 'pl', 'ps', 'pt', 'ro', 'ru', 'sd', 'si', 'sk', 'sl', 'so', 'sq', 'sr', 'ss', 'su', 'sv', 'sw', 'ta', 'th', 'tl', 'tn', 'tr', 'uk', 'ur', 'uz', 'vi', 'wo', 'xh', 'yi', 'yo', 'zh', 'zu'])


In [72]:
tokenizer_inf.src_lang = "pt"
tokenizer_inf.tgt_lang = "es"
forced_bos = tokenizer_inf.get_lang_id("es")  # id para 'es'

# Escribir ejemplos de prueba

In [73]:
pt_texts = [
    # 1. Frase informativa
    "A Comissão Europeia lançou um programa para apoiar as pequenas e médias empresas.",
    # 2. Oración con términos técnicos
    "Os sensores LiDAR são amplamente utilizados em veículos autónomos para detecção de obstáculos.",
    # 3. Texto jurídico
    "Segundo o artigo 15.º do regulamento, os Estados-Membros devem apresentar relatórios anuais até 31 de março.",
    # 4. Expresión coloquial
    "Eles estavam a brincar no parque quando começou a chover torrencialmente.",
    # 5. Oración con números y fechas
    "A conferência terá lugar no dia 12 de outubro de 2023, das 09:00 às 17:30.",
    # 6. Texto médico
    "O paciente apresenta sintomas de febre, tosse seca e dificuldade respiratória.",
    # 7. Frase promocional
    "A nova coleção de verão já está disponível nas lojas e no site oficial.",
    # 8. Oración con locuciones adverbiales
    "No entanto, apesar das dificuldades, a equipa manteve-se otimista até ao fim.",
    # 9. Texto financiero
    "O BCE elevou as taxas de juro em 0,25 pontos percentuais para conter a inflação.",
    # 10. Mensaje de redes sociales
    "Alguém viu o meu novo vídeo no YouTube sobre receitas saudáveis? Dêem uma olhada!"
]


# Tokenizar inputs

In [74]:
inputs = tokenizer_inf(
    pt_texts,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=128
)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generar traducciones

In [75]:
outputs = model_inf.generate(
    **inputs,
    forced_bos_token_id=forced_bos,
    max_new_tokens=60,
    num_beams=4,
    early_stopping=True
)

# Decodificar y mostrar resultados de traducciones

In [77]:
translations = tokenizer_inf.batch_decode(outputs, skip_special_tokens=True)
for src, tgt in zip(pt_texts, translations):
    print(f"PT ➞ {src}")
    print(f"ES ➞ {tgt}\n")

PT ➞ A Comissão Europeia lançou um programa para apoiar as pequenas e médias empresas.
ES ➞ La Comisión Europea ha lanzado un programa para apoyar a las pequeñas y medianas empresas.

PT ➞ Os sensores LiDAR são amplamente utilizados em veículos autónomos para detecção de obstáculos.
ES ➞ Los sensores LiDAR se utilizan ampliamente en vehículos autónomos para detectar obstáculos.

PT ➞ Segundo o artigo 15.º do regulamento, os Estados-Membros devem apresentar relatórios anuais até 31 de março.
ES ➞ De conformidad con el artículo 15 del Reglamento, los Estados miembros presentarán informes anuales antes del 31 de marzo.

PT ➞ Eles estavam a brincar no parque quando começou a chover torrencialmente.
ES ➞ Ellos estaban jugando en el parque cuando comenzó a lluviar torrencialmente.

PT ➞ A conferência terá lugar no dia 12 de outubro de 2023, das 09:00 às 17:30.
ES ➞ La conferencia tendrá lugar el 12 de octubre de 2023, de las 09:00 a las 17:30 horas.

PT ➞ O paciente apresenta sintomas de feb